# Прямые методы решения линейных систем

## Метод Гаусса с выбором главного элемента по столбцу

При написании программы нельзя использовать готовые функции для матричного умножения и обращения матриц кроме случаев,когда это явно описано.

Напишите программу для решения линейной системы $$Ax = b$$ методом Гаусса с выбором главного элемента по столбцу. Требования к программе:
1. Программа должна содержать функцию, принимающую на вход матрицу $A$ и правую часть $b$.
2. Внутри функции нужно сначала вычислить матрицы $L$, $U$ и матрицу перестановки $P$, соответствующую выбору главного элемента по столбцу (т.е. перестановке строк): $PA = LU$.
3. После этого в функции нужно решить системы с треугольными матрицами с помощью прямой и обратной подстановок.
4. Функция должна возвращать матрицы и вектор решения: $L, U, P, x$.
5. Программа должна вызывать реализованную фукнцию для какой-то матрицы и правой части и выводить норму разницы между полученным решением, и решением, которое возвращает готовая библиотечная функция, например, $numpy.linalg.solve$.

In [1]:
import numpy as np
from scipy import linalg

In [2]:
#function to solve system with triangle matrix T, vector b, flag: 'l' - lower, 'u' - upper triangle matrix
def solve_triang(T, b, flag):
    n = b.shape[0] # size of system
    solution = np.zeros(n)
    
    if (flag == 'l'):
        for k in range(n):
            solution[k] = b[k]
            for i in range(k):
                solution[k] -= solution[i] * T[k][i]
            solution[k] /= T[k][k]
    else:
        for k in range(n - 1, -1, -1):
            solution[k] = b[k]
            for i in range(n - 1, k, -1):
                solution[k] -= solution[i] * T[k][i]
            solution[k] /= T[k][k]
        
    return solution

In [3]:
#function solving linear system Ax = b using LU decomposition with pivoting
def solve_linear_system(A, b):
    #find decomposition of matrix A in a form PA = LU
    size = A.shape[0]
    U = np.copy(A)
    L = np.eye(size)
    P = np.eye(size)
    for k in range(size - 1):
        index = np.argmax(np.abs(U[k:size, k])) + k
        
        #swap lines in U
        tmp = np.copy(U[k, k:size])
        U[k, k:size] = np.copy(U[index, k:size])
        U[index, k:size] = np.copy(tmp)
        
        #swap lines in L
        tmp = np.copy(L[k, 0:k])
        L[k, 0:k] = np.copy(L[index, 0:k])
        L[index, 0:k] = np.copy(tmp)
        
        #swap lines in P
        tmp = np.copy(P[k, :])
        P[k, :] = np.copy(P[index, :])
        P[index, :] = np.copy(tmp)
        
        for j in range(k + 1, size):
            L[j, k] = U[j, k] / U[k, k]
            U[j, k:size] = U[j, k:size] - L[j, k] * U[k, k:size]
    
    #we have PA = LU, the system is Ax = b <=> system LUx = Pb
    #let Ux = y, Pb = b_new => let's solve Ly = b_new, than Ux = y
    
    #find b_new
    b_new = np.zeros(size)
    for i in range(size):
        for j in range(size):
            if (P[i, j] == 1):
                b_new[i] = b[j]
                
    #solving Ly = b_new
    y = solve_triang(L, b_new, 'l')
    
    #solving Ux = y
    x = solve_triang(U, y, 'u')
    
    return L, U, P, x

In [4]:
#creating a random matrix and random vector
size = 4
A = np.random.rand(size, size)
b = np.array([np.random.rand() for i in range(size)])

#solving system Ax = b with implemented function and with numpy.linalg.solve
L_my, U_my, P_my, x_my = solve_linear_system(A, b)
x_numpy = np.linalg.solve(A, b)

print("Solution with numpy.linalg.solve: {}".format(x_numpy))
print("Solution with implemented function using LU decomposition: {}".format(x_my))
print("Euclidean norm of the difference of two solutions: {}".format(np.linalg.norm(x_my - x_numpy, 2)))

Solution with numpy.linalg.solve: [-35.54604675   2.71370168  27.01263358 -16.35184099]
Solution with implemented function using LU decomposition: [-35.54604675   2.71370168  27.01263358 -16.35184099]
Euclidean norm of the difference of two solutions: 3.34080552873e-13
